In [2]:
import json
import string
import random
import re
import urllib.request
import numpy as np
import bs4 as bs
import nltk
import re
import unicodedata
import stanza
import spacy_stanza


###Corpus

In [3]:
stanza.download("es")
nlp = spacy_stanza.load_pipeline("es")

2021-09-23 01:56:56 INFO: Downloading default packages for language: es (Spanish)...
2021-09-23 01:56:58 INFO: File exists: /root/stanza_resources/es/default.zip.
2021-09-23 01:57:04 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-09-23 01:57:04 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2021-09-23 01:57:04 INFO: Use device: cpu
2021-09-23 01:57:04 INFO: Loading: tokenize
2021-09-23 01:57:04 INFO: Loading: mwt
2021-09-23 01:57:04 INFO: Loading: pos
2021-09-23 01:57:05 INFO: Loading: lemma
2021-09-23 01:57:05 INFO: Loading: depparse
2021-09-23 01:57:05 INFO: Loading: ner
2021-09-23 01:57:06 INFO: Done loading processors!


In [4]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Jorge_Luis_Borges')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [5]:
article_text

"\njorge francisco isidoro luis borges (buenos aires, 24 de agosto de 1899-ginebra, 14 de junio de 1986), más conocido como jorge luis borges, fue un destacado escritor de cuentos, poemas y ensayos argentino, extensamente considerado una figura clave tanto para la literatura en habla hispana como para la literatura universal.[2]\u200b también fue bibliotecario, profesor, conferencista, crítico literario y traductor. sus dos libros más conocidos, ficciones y el aleph, publicados en los años cuarenta, son recopilaciones de cuentos conectados por temas comunes de forma fantástica; como los sueños, los laberintos, las bibliotecas, los espejos, los autores ficticios y las mitologías europeas (como la griega y la nórdica), con argumentos que exploran ideas filosóficas relacionadas, por ejemplo, con la memoria, la eternidad, la posmodernidad y la metaficción.[3]\u200b las obras de borges han contribuido ampliamente a la literatura filosófica, al género fantástico y al posestructuralismo. segú

In [6]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 81639


In [7]:
text = re.sub(r'\[[0-9]*\]', ' ', article_text)
text = re.sub(r'\s+', ' ', text)

In [8]:
text

" jorge francisco isidoro luis borges (buenos aires, 24 de agosto de 1899-ginebra, 14 de junio de 1986), más conocido como jorge luis borges, fue un destacado escritor de cuentos, poemas y ensayos argentino, extensamente considerado una figura clave tanto para la literatura en habla hispana como para la literatura universal. \u200b también fue bibliotecario, profesor, conferencista, crítico literario y traductor. sus dos libros más conocidos, ficciones y el aleph, publicados en los años cuarenta, son recopilaciones de cuentos conectados por temas comunes de forma fantástica; como los sueños, los laberintos, las bibliotecas, los espejos, los autores ficticios y las mitologías europeas (como la griega y la nórdica), con argumentos que exploran ideas filosóficas relacionadas, por ejemplo, con la memoria, la eternidad, la posmodernidad y la metaficción. \u200b las obras de borges han contribuido ampliamente a la literatura filosófica, al género fantástico y al posestructuralismo. según mar

In [9]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 81031


In [10]:
corpus = nltk.sent_tokenize(text)
words = nltk.word_tokenize(text)

In [11]:
corpus[:10]

[' jorge francisco isidoro luis borges (buenos aires, 24 de agosto de 1899-ginebra, 14 de junio de 1986), más conocido como jorge luis borges, fue un destacado escritor de cuentos, poemas y ensayos argentino, extensamente considerado una figura clave tanto para la literatura en habla hispana como para la literatura universal.',
 '\u200b también fue bibliotecario, profesor, conferencista, crítico literario y traductor.',
 'sus dos libros más conocidos, ficciones y el aleph, publicados en los años cuarenta, son recopilaciones de cuentos conectados por temas comunes de forma fantástica; como los sueños, los laberintos, las bibliotecas, los espejos, los autores ficticios y las mitologías europeas (como la griega y la nórdica), con argumentos que exploran ideas filosóficas relacionadas, por ejemplo, con la memoria, la eternidad, la posmodernidad y la metaficción.',
 '\u200b las obras de borges han contribuido ampliamente a la literatura filosófica, al género fantástico y al posestructuralis

In [12]:
words[:20]

['jorge',
 'francisco',
 'isidoro',
 'luis',
 'borges',
 '(',
 'buenos',
 'aires',
 ',',
 '24',
 'de',
 'agosto',
 'de',
 '1899-ginebra',
 ',',
 '14',
 'de',
 'junio',
 'de',
 '1986']

In [13]:
print("Vocabulario:", len(words))

Vocabulario: 15146


###Preprocesamiento

In [23]:
def preprocess_clean_text(text):    
    # sacar tildes de las palabras
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    # quitar números
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntiación
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

In [24]:
def get_processed_text(document):
    tokens = nlp(preprocess_clean_text(document.lower()))
    return [token.lemma_ for token in tokens]

###Vectorizacion

In [16]:
stop_words = []
with open('stop_words_spanish.txt') as file:
  while True:
    word = file.readline()
    if word == "":
      break
    word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    stop_words.append(word.rstrip())
  

In [18]:
stop_words[:10]

['algun',
 'alguna',
 'algunas',
 'alguno',
 'algunos',
 'ambos',
 'ampleamos',
 'ante',
 'antes',
 'aquel']

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words=stop_words)

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "mmm, perdon no me se esa de Borges"
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

In [21]:
import sys
!{sys.executable} -m pip install gradio --quiet

In [ ]:
import gradio as gr

def bot_response(human_text):
    print(human_text)
    return generate_response(human_text.lower(), corpus)

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://33489.gradio.app
Interface loading below...


borges



/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afirmar', 'agregar', 'amplear', 'asegurar', 'atra', 'comentar', 'considerar', 'contar', 'cosa', 'creer', 'cualquiera', 'deber', 'dejar', 'demar', 'encontrar', 'entrar', 'er', 'existir', 'explicar', 'gonar', 'grande', 'indicar', 'informar', 'llegar', 'mencio